In [1]:
import numpy as np
from scipy import *
import os
import h5py
import keras as k
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD, adam
from keras.models import Sequential
from keras.losses import categorical_crossentropy
from sklearn.utils import shuffle
from keras.utils import np_utils
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import data, exposure

Using TensorFlow backend.


In [2]:
# Setting directory paths 
root_dir = os.getcwd()

# Loading data
data = h5py.File(r'C:\\Users\\fires\\Desktop\\CSIRO\\thursday\\Logistic Regression\\data.h5', 'r')

images = np.asarray(data['images'])
labels = data['labels']

In [3]:
# Define function to crop off 'cut' number of pixels on both sides on both axixs
def crop_center(img,cut):
    y = np.shape(img)[1]
    x = np.shape(img)[2]
    if x != y:
        print ("The image is not a perfect sqaure. This is bad. Fix it ")
        
    start = cut
    end = x-cut
    return img[:, start:end,start:end]

In [4]:
# Defining rescaling function
def rescale_imgs(images, factor):
    count =  0
    for i in range(0 ,images.shape[0]):
        img_scaled = expand_dims(rescale(images[i, :, :], 1.0 / float(factor)), axis = 0)
        if count == 0:
            imgs = img_scaled
        else:
            imgs = np.vstack((imgs, img_scaled))
        count += 1
                             
    return imgs

In [5]:
# Reducing Image Size
images_cut = crop_center(images,66)
images_cut = np.asarray(images_cut)


In [6]:
hog_images  = []

for i, image in enumerate(images_cut):
    fd = hog(images_cut[i, :, :], orientations=8, pixels_per_cell=(8, 8),
                        cells_per_block=(1, 1), feature_vector=True)
    hog_images.append(fd)

C:\Users\fires\Anaconda3\envs\CSIRO\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


In [7]:
# balance classes
def bal_classes(images, labels):
    class1 = []
    class2 = []
    
    for i in range(labels.shape[0]):
        if labels[i] == False:
            class1.append(i)
    
    for i in range(labels.shape[0]):
        if labels[i] == True:
            class2.append(i)
    
    if len(class2) > len(class1):
        class2 = np.asarray(class2[:len(class1)])
    
    elif len(class2) < len(class1):
        class1 = np.asarray(class1[:len(class2)])
        
    images1 = images[class1, :] 
    images2 = images[class2, :]
    labels1 = labels[class1,]
    labels2 = labels[class2,]
    
    print (labels2.shape)
    
    images_st = np.vstack((images1, images2))
    labels_st =  np.concatenate((labels1, labels2), axis=0)              
        
    images_s, labels_s = shuffle(np.asarray(images_st), np.asarray(labels_st), random_state = 0)    
        
    return images_s, labels_s

In [8]:
images_p, labels_p = bal_classes(np.asarray(hog_images), np.asarray(labels))

NameError: name 'hog_img' is not defined

In [ ]:
#imgs = np.reshape(images_p, (-1, 135, 135))

#for n in range(0, imgs.shape[0]):
   #plt.imshow(imgs[n,:,:], cmap='gray', shape=(135, 135))
   #print (n)
   #print (normalized_labels[n])
   #plt.show()

In [ ]:
# Split data into training and testing sets

def format_data(images, labels, ratio = 0.8):
    split = round(ratio*images.shape[0])
    
    train_x = images[ :split, :]
    test_x = images[split:, :]
    train_y = labels[ :split]
    test_y = labels[split:]
    
    return train_x, test_x, train_y, test_y


train_x, test_x, train_y, test_y = format_data(images_p, labels_p)

In [ ]:
# Convert class vectors to binary class matrices
train_y = np_utils.to_categorical(train_y, 2)
test_y = np_utils.to_categorical(test_y, 2)

In [ ]:
# Define model attributes
batch_size = 20
nb_classes = output_dims = 2
nb_epoch = 50
input_dim = 21632

model = Sequential()
model.add(Dense(output_dims, input_dim = input_dim, activation='sigmoid'))



In [ ]:
adam = adam(lr=0.1)
model.compile(optimizer="adam", loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(train_x, train_y, epochs=nb_epoch, batch_size=batch_size, 
          validation_data=(test_x, test_y))

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model_hog.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_hog.h5")
print("Saved model to disk")

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
for layer in model.layers:
    weights = layer.get_weights()

wt = np.reshape(weights[0], (135, 135, 2))
    
for n in range(0, wt.shape[2]):
   plt.imshow(wt[:,:,n], cmap='gray', shape=(135, 135))
   print (n)
   plt.show()

print ("Dense to output layer weights " + str(weights[1]))